In [1]:
from vietadminunits import parse_address
import pandas as pd
import re
from unidecode import unidecode

# Demo

In [2]:
parse_address('Ngã Tư Sở Market, Đường Nguyễn Trãi, Thượng Đình Số nhà 28A, Ngõ 72 Nguyễn Trãi , , Phường Thượng Đình, Quận Thanh Xuân , Hà Nội Quận Thanh Xuân')

AdministrativeUnit:
Attribute                      | Value                         
--------------------------------------------------------------
province                       | Hà Nội                        
district                       | Thanh Xuân                    
ward                           | Thượng Đình                   
--------------------------------------------------------------
long_province                  | Thành phố Hà Nội              
long_district                  | Quận Thanh Xuân               
long_ward                      | Phường Thượng Đình            
--------------------------------------------------------------
short_district                 | Thanh Xuân                    
short_ward                     | Thượng Đình                   
--------------------------------------------------------------
province_english               | Ha Noi                        
district_english               | Thanh Xuan                    
ward_english            

In [3]:
parse_address('Cuối đường kiều mai, dự án bệnh viện hạnh phúc  phường phú diễn quận bắc từ liêm hà nội')

AdministrativeUnit:
Attribute                      | Value                         
--------------------------------------------------------------
province                       | Hà Nội                        
district                       | Bắc Từ Liêm                   
ward                           | Phú Diễn                      
--------------------------------------------------------------
long_province                  | Thành phố Hà Nội              
long_district                  | Quận Bắc Từ Liêm              
long_ward                      | Phường Phú Diễn               
--------------------------------------------------------------
short_district                 | Bắc Từ Liêm                   
short_ward                     | Phú Diễn                      
--------------------------------------------------------------
province_english               | Ha Noi                        
district_english               | Bac Tu Liem                   
ward_english            

In [4]:
parse_address('03 Nguyễn Cao Luyện, An Hải Bắc, Sơn Trà, Đà Nẵng 550000, Việt Nam')

AdministrativeUnit:
Attribute                      | Value                         
--------------------------------------------------------------
province                       | Đà Nẵng                       
district                       | Sơn Trà                       
ward                           | An Hải Bắc                    
--------------------------------------------------------------
long_province                  | Thành phố Đà Nẵng             
long_district                  | Quận Sơn Trà                  
long_ward                      | Phường An Hải Bắc             
--------------------------------------------------------------
short_district                 | Sơn Trà                       
short_ward                     | An Hải Bắc                    
--------------------------------------------------------------
province_english               | Da Nang                       
district_english               | Son Tra                       
ward_english            

In [5]:
parse_address('3c60/10 chợ cầu xấng, xã phạm văn hai, huyện bình chánh HCM')

AdministrativeUnit:
Attribute                      | Value                         
--------------------------------------------------------------
province                       | Hồ Chí Minh                   
district                       | Bình Chánh                    
ward                           | Phạm Văn Hai                  
--------------------------------------------------------------
long_province                  | Thành phố Hồ Chí Minh         
long_district                  | Huyện Bình Chánh              
long_ward                      | Xã Phạm Văn Hai               
--------------------------------------------------------------
short_district                 | Bình Chánh                    
short_ward                     | Phạm Văn Hai                  
--------------------------------------------------------------
province_english               | Ho Chi Minh                   
district_english               | Binh Chanh                    
ward_english            

In [6]:
parse_address('ngõ 154 Đình Thôn, phường Mỹ Đình, Quận Nam Từ Liêm, HN')

AdministrativeUnit:
Attribute                      | Value                         
--------------------------------------------------------------
province                       | Hà Nội                        
district                       | Nam Từ Liêm                   
ward                           |                               
--------------------------------------------------------------
long_province                  | Thành phố Hà Nội              
long_district                  | Quận Nam Từ Liêm              
long_ward                      |                               
--------------------------------------------------------------
short_district                 | Nam Từ Liêm                   
short_ward                     |                               
--------------------------------------------------------------
province_english               | Ha Noi                        
district_english               | Nam Tu Liem                   
ward_english            

In [7]:
parse_address('ngõ 154 Đình Thôn, phường Mỹ Đình, Quận Nam Từ Liêm, HN', level=1)

AdministrativeUnit:
Attribute                      | Value                         
--------------------------------------------------------------
province                       | Hà Nội                        
district                       |                               
ward                           |                               
--------------------------------------------------------------
long_province                  | Thành phố Hà Nội              
long_district                  |                               
long_ward                      |                               
--------------------------------------------------------------
short_district                 |                               
short_ward                     |                               
--------------------------------------------------------------
province_english               | Ha Noi                        
district_english               |                               
ward_english            

In [8]:
# parse_address('Ha Noi, Gia Lam, Yen Vien')

# Check wrong or missing in bulk

In [9]:
df = pd.read_csv('../data/input/danhmuchanhchinhgso.gov.vn.csv')

In [10]:
df

,long_province,long_district,long_ward,ward_level
0,Thành phố Hà Nội,Quận Ba Đình,Phường Phúc Xá,Phường
1,Thành phố Hà Nội,Quận Ba Đình,Phường Trúc Bạch,Phường
2,Thành phố Hà Nội,Quận Ba Đình,Phường Vĩnh Phúc,Phường
3,Thành phố Hà Nội,Quận Ba Đình,Phường Cống Vị,Phường
4,Thành phố Hà Nội,Quận Ba Đình,Phường Liễu Giai,Phường
...,...,...,...,...
10542,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An Đông,Xã
10543,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An,Xã
10544,Tỉnh Cà Mau,Huyện Ngọc Hiển,Thị trấn Rạch Gốc,Thị trấn
10545,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Tân Ân,Xã


In [13]:
df['district_address'] = df['long_district'] + ', ' + df['long_province']
district_addresses = df['district_address'].tolist()

for address in district_addresses:
    try:
        unit = parse_address(address, level=2)
    except:
        print(address)
    if unit.province == None:
        print(address)
    elif unit.district == None:
        print(address)

In [14]:
df['ward_address'] = df['long_ward'] + ', ' + df['long_district'] + ', ' + df['long_province']
ward_addresses = df['ward_address'].dropna().tolist()
miss = []
for ward_address in ward_addresses:
    try:
        unit = parse_address(address, level=3)
    except:
        print(address)
        # print(unit)
        miss.append((address, unit))
    if unit.province == None:
        print(address)
        # print(unit)
        miss.append((address, unit))
    elif unit.district == None:
        print(address)
        # print(unit)
        miss.append((address, unit))
    elif unit.ward == None:
        print(address)
        # print(unit)
        miss.append((address, unit))

In [17]:
parse_address('Xã Tống Trân, Huyện Phù Cừ, Tỉnh Hưng Yên')

AdministrativeUnit:
Attribute                      | Value                         
--------------------------------------------------------------
province                       | Hưng Yên                      
district                       | Phù Cừ                        
ward                           | Tống Trân                     
--------------------------------------------------------------
long_province                  | Tỉnh Hưng Yên                 
long_district                  | Huyện Phù Cừ                  
long_ward                      | Xã Tống Trân                  
--------------------------------------------------------------
short_district                 | Phù Cừ                        
short_ward                     | Tống Trân                     
--------------------------------------------------------------
province_english               | Hung Yen                      
district_english               | Phu Cu                        
ward_english            